### ¿Qué es un corrector gramatical?

Es un sistema de revisión de textos, capaz de implementar reglas de análisis y corrección de textos basados en el idioma específico con el que se está trabajando.

Capacidad de corrección:

Entre otras cosas, un correcto gramatical puede detectar lo siguiente:

- Faltas de ortografía en palabras comunes  (Por fabor  →  Por favor) 

- Combinación incorrecta de singulares y plurales  (Mi casas  →  Mi casa)

- Palabras repetidas (Le dije eso eso ayer  →  Le dije eso ayer)

- Mayúsculas en lugares incorrectos  (LA tarDe  →  La tarde)

- Problemas con verbos auxiliares (Debes pagado  →  Debes pagar)

Entre otras aplicaciones




In [66]:
# TAREA 01: Construir un corrector gramatical que revise en un texto errores como los que se detallan a continuación. Imprimir 
# el texto corregido marcando los cambios entre corchetes “[]” Mostrar la cantidad y tipo de correcciones realizadas:

# Tips: 
#- Se debe utilizar un método por cada caso

#- La tarea se trabajará directamente en un Jupyter Notebook y se deberá de poder ejecutar con la instalación realizada en 
# clase, y en caso de usar alguna librería adicional, especificarlo en el mismo notebook


In [67]:
import spacy
from spacy import displacy

nlp = spacy.load("es_core_news_md")

In [68]:
def corrector_ortografico(texto):
    """Esta función verifica la ortografía de las palabras en un texto y corrige errores.

    Args:
        texto: El texto que se va a revisar.

    Returns:
        El texto con los errores corregidos y el número de correcciones.
    """

    # Crear un diccionario de palabras comúnmente mal escritas.
    palabras_incorrectas = {
        "q": "que",
        "x": "por",
        "cmo": "cómo",
        "dnd": "dónde",
        "sq": "es que",
        "sta": "esta",
        "msj": "mensaje",
        "aki": "aquí",
        "tmb": "también",
        "qro": "quiero",
        "pq": "porque"
    }

    # Crear un documento spaCy a partir del texto.
    doc = nlp(texto)

    texto_corregido = ""
    num_correcciones = 0
    for token in doc:
        if token.text.lower() in palabras_incorrectas:
            texto_corregido += "[" + palabras_incorrectas[token.text.lower()] + "]"
            num_correcciones += 1
        else:
            texto_corregido += token.text

        texto_corregido += " "

    return texto_corregido, num_correcciones


texto = "Hola q tal? Quiero ir aki pq tmb sta el msj dnd me dices sq viste algo x allá."

texto_corregido, num_correcciones = corrector_ortografico(texto)

print("Texto original: {}".format(texto))
print("Texto corregido: {}".format(texto_corregido))
print("Número de correcciones: {}".format(num_correcciones))

Texto original: Hola q tal? Quiero ir aki pq tmb sta el msj dnd me dices sq viste algo x allá.
Texto corregido: Hola [que] tal ? Quiero ir [aquí] [porque] [también] [esta] el [mensaje] [dónde] me dices [es que] viste algo [por] allá . 
Número de correcciones: 9


In [69]:
# Caso 2: Verificar que no exista una palabra en singular, seguida de una en plural y al revés

In [70]:
def correct_number_mismatches(doc):
    """Esta función verifica que no exista una palabra en singular, seguida de una en plural y al revés.

    Args:
        doc: El texto que se va a revisar.

    Returns:
        El texto con los errores corregidos y el número de correcciones.
    """
    correcciones = []
    for i, token in enumerate(doc[:-1]):  # Usamos doc[:-1] para evitar acceder fuera de rango
        next_tok = doc[i + 1]

        # Corregimos las comparaciones
        if token.tag_ == 'DET' and token.text[-1] == 's' and next_tok.tag_ == 'NOUN' and next_tok.text[-1] != 's':
            correcciones.append((next_tok.text, next_tok.text + 's'))
        elif token.tag_ == 'DET' and token.text[-1] != 's' and next_tok.tag_ == 'NOUN' and next_tok.text[-1] == 's':
            correcciones.append((next_tok.text, next_tok.text[:-1]))

    return correcciones


def apply_corrections(text, correcciones):
    """Esta función aplica las correcciones sugeridas por la función anterior.

    Args:
        text: El texto que se va a revisar.
        correcciones: La lista de correcciones sugeridas.

    Returns:
        El texto con los errores corregidos.
    """
    for original, corrected in correcciones:
        text = text.replace(original, corrected)
    return text


text = "Los coche son rojos. Mi amigos es amable."
doc = nlp(text)
correcciones = correct_number_mismatches(doc)
texto_corregido = apply_corrections(text, correcciones)
print("Texto original:", text)
print('Correcciones sugeridas: ', correcciones)
print("Texto corregido:", texto_corregido)

Texto original: Los coche son rojos. Mi amigos es amable.
Correcciones sugeridas:  [('coche', 'coches'), ('amigos', 'amigo')]
Texto corregido: Los coches son rojos. Mi amigo es amable.


In [71]:
# Caso 3: Verificar que no exista la misma palabra dos veces seguidas

In [72]:
def no_repetir_palabras(texto):
    """Esta función verifica que no exista la misma palabra repetida dos veces seguidas en un texto.

    Args:
        texto: El texto a revisar.

    Returns:
        El texto con las correcciones y el número de correcciones.
    """

    # Crear un documento spaCy a partir del texto.
    doc = nlp(texto)

    texto_corregido = ""
    num_correcciones = 0

    prev_token = None
    for token in doc:
        if prev_token is not None and token.text.lower() == prev_token.text.lower():
            num_correcciones += 1
            continue
        else:
            texto_corregido += token.text + " "
            prev_token = token

    return texto_corregido.strip(), num_correcciones


texto = "Estamos probando probando que no haya palabras palabras repetidas en este este texto"
texto_corregido, num_correcciones = no_repetir_palabras(texto)
print("Texto original: {}".format(texto))
print("Texto corregido: {}".format(texto_corregido))
print("Número de correcciones: {}".format(num_correcciones))

Texto original: Estamos probando probando que no haya palabras palabras repetidas en este este texto
Texto corregido: Estamos probando que no haya palabras repetidas en este texto
Número de correcciones: 3


In [73]:
# Caso 4: Verificar que la forma de un token no mezcle mayúsculas, minúsculas y dígitos, a menos que se trate de todo en mayúsculas para marcar siglas 

In [86]:
import spacy

nlp = spacy.load("es_core_news_md")

def check_token_case(token):
    """Verifica si el token cumple con la regla de mayúsculas y minúsculas.

    Args:
        token (Token): El token a verificar.

    Returns:
        bool: True si el token cumple con la regla, False si no.
    """
    text = token.text

    if text.isupper():  # Todo en mayúsculas (siglas están permitidas)
        return True
    elif text.islower():  # Todo en minúsculas
        return True
    elif text.isdigit():  # Solo dígitos
        return True
    else:
        # Verificar que no tenga combinación de mayúsculas, minúsculas y dígitos
        if (any(char.islower() for char in text) and any(char.isupper() for char in text)) or \
                (any(char.islower() for char in text) and any(char.isdigit() for char in text)) or \
                (any(char.isdigit() for char in text) and any(char.isupper() for char in text)):
            return False
    return True  # Cualquier otra combinación no especificada se considera válida

def correct_token_case(text):
    """Corrige los tokens del texto que no cumplan con la regla de mayúsculas y minúsculas.

    Args:
        text (str): El texto a corregir.

    Returns:
        str: El texto corregido.
    """
    doc = nlp(text)
    corrected_tokens = []

    for token in doc:
        if not check_token_case(token):
            corrected_tokens.append(token.text.lower())  # Convertimos a minúsculas si no cumple
        else:
            corrected_tokens.append(token.text)

    return " ".join(corrected_tokens)

def capitalize_after_period(text):
    """Corrige un texto para que después de un punto y la primera palabra empiecen con mayúscula.

    Args:
        text (str): El texto a corregir.

    Returns:
        str: El texto corregido.
    """
    # Primero, aseguramos que la primera palabra esté en mayúsculas
    text = text[0].upper() + text[1:]

    # Luego, aseguramos que cualquier palabra después de un punto esté en mayúsculas
    sentences = text.split(". ")
    sentences = [s[0].upper() + s[1:] if len(s) > 0 else s for s in sentences]

    return ". ".join(sentences)

texto_mayus = "los gatOS son animales. son muy independientes. a veces son cariñosos"
texto_corregido = capitalize_after_period(correct_token_case(texto_mayus))

print("Texto original:", texto_mayus)
print("Texto corregido:", texto_corregido)


Texto original: los gatOS son animales. son muy independientes. a veces son cariñosos
Texto corregido: Los gatos son animales . Son muy independientes . A veces son cariñosos


In [75]:
# Caso 5: Identificar cuando se escriben 2 “NOUNS” seguidos, pero no se trata de un NOMBRE propio

In [77]:
def check_noun_noun_rule(text):
    """Esta función verifica que no exista dos sustantivos seguidos que no sean nombres propios.

    Args:
        text: El texto a revisar.

    Returns:
        True si no hay dos sustantivos seguidos que no sean nombres propios, False si sí hay.
    """
    doc = nlp(text)
    for i, token in enumerate(doc[:-1]):
        next_tok = doc[i + 1]

        if (token.tag_ == 'PROPN' or token.tag_ == 'NOUN') and (
                next_tok.tag_ == 'PROPN' or next_tok.tag_ == 'NOUN') and not next_tok.text.istitle():
            return False
    return True


def detectar_dos_nouns_seguidos(texto):
    """Esta función verifica que no exista dos sustantivos seguidos que no sean nombres propios.

    Args:
        text: El texto a revisar.

    Returns:
        Una lista de los segundos sustantivos de los pares encontrados y el texto corregido.
    """
    if not check_noun_noun_rule(texto):
        doc = nlp(texto)

        # Lista de pares de sustantivos encontrados
        pares_sustantivos = []

        token_ant = None
        for token in doc:
            if token_ant is not None and (token_ant.tag_ == 'PROPN' or token_ant.tag_ == 'NOUN') and (
                    token.tag_ == 'PROPN' or token.tag_ == 'NOUN') and not token.text.istitle():
                pares_sustantivos.append((token_ant.i, token.i))
            token_ant = token

        # Obtener solo los segundos sustantivos de los pares
        segundos_sustantivos = [doc[pair[1]].text for pair in pares_sustantivos]

        # Construir texto corregido usando los pares identificados
        texto_corregido = []
        skip_next = False
        for i, token in enumerate(doc):
            if any([pair[1] == i for pair in pares_sustantivos]):
                skip_next = True
            if skip_next:
                skip_next = False
                continue
            texto_corregido.append(token.text)

        return segundos_sustantivos, " ".join(texto_corregido)

    else:
        return [], texto


texto = "El perro gato juegan en el parque"
segundos_sustantivos, texto_corregido = detectar_dos_nouns_seguidos(texto)
if segundos_sustantivos:
    print("Texto original: {}".format(texto))
    print("Segundos sustantivos encontrados:", segundos_sustantivos)
    print("Texto corregido: {}".format(texto_corregido))
else:
    print("El texto no tiene dos sustantivos seguidos que no son nombres propios.")


Texto original: El perro gato juegan en el parque
Segundos sustantivos encontrados: ['gato']
Texto corregido: El perro juegan en el parque
